## Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import callbacks

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
print(application_df.columns)

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')


In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [20]:
# Look at APPLICATION_TYPE value counts for binning
application_count=application_df["APPLICATION_TYPE"].value_counts()
print(application_count)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [21]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

cutoff = 500
application_types_to_replace = application_count[application_count < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Look at CLASSIFICATION value counts for binning
Classification_count = application_df["CLASSIFICATION"].value_counts()
print(Classification_count)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [23]:
# You may find it helpful to look at CLASSIFICATION value counts >1
Classification_count_greaterthan_1 = Classification_count[Classification_count > 1 ]
print(Classification_count_greaterthan_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(Classification_count_greaterthan_1[Classification_count_greaterthan_1 <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 78)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
X_train_scaled.shape

(25724, 69)

## Compile, Train and Evaluate the Model**

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_features = len( X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                5600      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,061
Trainable params: 8,061
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create callback to save weights every 5 epochs
cb = tf.keras.callbacks.ModelCheckpoint(filepath='weights_{epoch:03d}.h5', save_weights_only=True, period=5)


In [31]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cb])



Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5773 - accuracy: 0.7201
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7322
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7327
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7338
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7348
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7347
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7350
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7349
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5452 - accuracy: 0.7363
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5572 - accuracy: 0.7250 - 507ms/epoch - 2ms/step
Loss: 0.5571949481964111, Accuracy: 0.7250145673751831


In [33]:
# Export our model to HDF5 file
nn.save('../AlphabetSoupCharity.h5')

## Optimize the Model

In [34]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# Create a function to build a Sequential model with randomized hyperparameter options within certain ranges defined below
def create_model(hp:kt.HyperParameters)->tf.keras.Model:
    nn = tf.keras.models.Sequential()

    # Allow keras tuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow keras tuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=10,
        max_value=100,
        step=10), activation=activation, input_dim=X_train.shape[1]))

    # Allow keras tuner to decide number of hidden layers
    for i in range(hp.Int('num_layers', 4, 8)):
        # Allow keras tuner to decide the number of neurons in the additional layers
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=10,
            max_value=100,
            step=10),
            activation=activation))
    
    # Add the final output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn

In [36]:
# Create a tuner with the goal of best accuracy within a short run of 20 epochs
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [37]:
# Run the keras tuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 07s]
val_accuracy: 0.7286297082901001

Best val_accuracy So Far: 0.7294460535049438
Total elapsed time: 00h 31m 17s


In [38]:

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 100,
 'num_layers': 8,
 'units_0': 50,
 'units_1': 90,
 'units_2': 90,
 'units_3': 70,
 'units_4': 70,
 'units_5': 80,
 'units_6': 30,
 'units_7': 50,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [39]:

# Create new model based on hyperparameters of best results from keras tuner
actvn = best_hyper.values['activation']
input = X_train.shape[1]

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=best_hyper.values['first_units'], activation=actvn, input_dim=input))

# Add the additional hidden layers
for i in range(best_hyper.values['num_layers']):
  unit = 'units_' + str(i)
  model.add(tf.keras.layers.Dense(units=best_hyper.values[unit], activation=actvn))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               7000      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 90)                4590      
                                                                 
 dense_10 (Dense)            (None, 90)                8190      
                                                                 
 dense_11 (Dense)            (None, 70)                6370      
                                                                 
 dense_12 (Dense)            (None, 70)                4970      
                                                                 
 dense_13 (Dense)            (None, 80)               

In [40]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create callback to save weights every 5 epochs
cb = tf.keras.callbacks.ModelCheckpoint(filepath='weights_{epoch:03d}.h5', save_weights_only=True, period=5)

# Create callback to stop if our model doesn't improve loss within 50 epochs & restore to best previous weights
es = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=50, restore_best_weights=True, verbose=1)

In [41]:
# Train the model with new settings
fit_model = model.fit(X_train_scaled, y_train, epochs=500, callbacks=[cb, es])


Epoch 1/500
804/804 [==============================] - 8s 7ms/step - loss: 0.5699 - accuracy: 0.7246
Epoch 2/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5556 - accuracy: 0.7291
Epoch 3/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5542 - accuracy: 0.7334
Epoch 4/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5517 - accuracy: 0.7332
Epoch 5/500
804/804 [==============================] - 6s 7ms/step - loss: 0.5502 - accuracy: 0.7339
Epoch 6/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5490 - accuracy: 0.7327
Epoch 7/500
804/804 [==============================] - 3s 3ms/step - loss: 0.5484 - accuracy: 0.7335
Epoch 8/500
804/804 [==============================] - 3s 4ms/step - loss: 0.5477 - accuracy: 0.7352
Epoch 9/500
804/804 [==============================] - 4s 6ms/step - loss: 0.5472 - accuracy: 0.7353
Epoch 10/500
804/804 [==============================] - 4s 5ms/step - loss: 0.5467 - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5627 - accuracy: 0.7280 - 540ms/epoch - 2ms/step
Loss: 0.5627173781394958, Accuracy: 0.7280466556549072


In [43]:

# Export our model to HDF5 file
model.save('AlphabetSoupCharity_Optimization.h5')